In [1]:
from pprint import pprint

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import datasets
import torch
import torch.nn
import torchmetrics.classification
from transformers import AutoImageProcessor, AutoModelForImageClassification

from armory.metrics.compute import BasicProfiler
from charmory.data import ArmoryDataLoader
from charmory.engine import EvaluationEngine
import charmory.evaluation as ev
from charmory.metrics.perturbation import PerturbationNormMetric
from charmory.model.image_classification import JaticImageClassificationModel
from charmory.perturbation import ArtEvasionAttack
from charmory.tasks.image_classification import ImageClassificationTask
from charmory.track import track_init_params, track_params
from charmory.utils import Unnormalize
import mlflow
from PIL import  Image
import os

In [2]:
model = JaticImageClassificationModel(
    track_params(AutoModelForImageClassification.from_pretrained)(
        "farleyknight-org-username/vit-base-mnist"
    ),
)
classifier = track_init_params(PyTorchClassifier)(
    model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(3, 224, 224),
    channels_first=True,
    nb_classes=10,
    clip_values=(-1, 1),
)

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [3]:
    dataset = datasets.load_dataset("mnist", split="test")
    processor = AutoImageProcessor.from_pretrained(
        "farleyknight-org-username/vit-base-mnist"
    )

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [4]:
batch_size = 16
def transform(sample):
        # Use the HF image processor and convert from BW To RGB
        sample["image"] = processor([img.convert("RGB") for img in sample["image"]])[
            "pixel_values"
        ]
        return sample

dataset.set_transform(transform)
dataloader = ArmoryDataLoader(dataset, batch_size=batch_size)

In [5]:
    attack = track_init_params(ProjectedGradientDescent)(
        classifier,
        batch_size=batch_size,
        eps=0.031,
        eps_step=0.007,
        max_iter=20,
        num_random_init=1,
        random_eps=False,
        targeted=False,
        verbose=False,
    )

In [6]:
evaluation = ev.Evaluation(
    name="mnist-vit-pgd",
    description="MNIST image classification using a ViT model and PGD attack",
    author="TwoSix",
    dataset=ev.Dataset(
        name="MNIST",
        x_key="image",
        y_key="label",
        test_dataloader=dataloader,
    ),
    model=ev.Model(
        name="ViT",
        model=classifier,
    ),
    perturbations={
        "benign": [],
        "attack": [
            ArtEvasionAttack(
                name="PGD",
                attack=attack,
                use_label_for_untargeted=False,
            ),
        ],
    },
    metric=ev.Metric(
        profiler=BasicProfiler(),
        perturbation={
            "linf_norm": PerturbationNormMetric(ord=torch.inf),
        },
        prediction={
            "accuracy": torchmetrics.classification.Accuracy(
                task="multiclass", num_classes=10
            ),
        },
    ),
)


In [7]:
export_every_n_batches = 10
num_batches = 10
task = ImageClassificationTask(
        evaluation,
        export_adapter=Unnormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        export_every_n_batches=export_every_n_batches,
    )
engine = EvaluationEngine(task, limit_test_batches=num_batches)

INFO: GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs


In [8]:
pprint(engine.run())

2024-01-03 22:11:56 20m53s WARNING  logging:callHandlers:1706 Experiment with name mnist-vit-pgd not found. Creating it.


2024/01/03 22:11:56 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/01/03 22:11:56 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/jonathan.prokos/.venv/armory-library/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/jonathan.prokos/.venv/armory-library/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
2024/01/03 22:13:03 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/01/03 22:13:03 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


{'compute': {'Avg. CPU time (s) for 1 executions of attack/perturbation': 45.86396195899579,
             'Avg. CPU time (s) for 1 executions of attack/perturbation/PGD': 45.86395349999657,
             'Avg. CPU time (s) for 1 executions of attack/predict': 0.8027910420059925,
             'Avg. CPU time (s) for 2 executions of benign/perturbation': 4.579997039400041e-07,
             'Avg. CPU time (s) for 2 executions of benign/predict': 0.8021256249994622},
 'metrics': {}}
